In [1]:
import gc
import os
from pathlib import Path
import random
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime,timedelta
import sklearn
import matplotlib.dates as mdates
%matplotlib inline
#from pyecharts.charts import Map,Geo
#from pyecharts import options as opts
#from pyecharts.globals import GeoType,RenderType
import json

import csv
#from plotly import tools, subplots
#import plotly.offline as py
#py.init_notebook_mode(connected=True)
#import plotly.express as px
#import plotly.io as pio
#pio.templates.default = "plotly_dark"
#import plotly.offline as pltoff
#from plotly.graph_objs import *
#pltoff.init_notebook_mode(connected=True)

# --- models ---
from sklearn import preprocessing
from sklearn.model_selection import KFold
#import lightgbm as lgb
#import xgboost as xgb
# --- setup ---
#pd.set_option('max_columns', 50)

In [2]:
US_data = pd.read_csv('time_series_covid19_confirmed_US.csv')
US_data.rename({'Province_State': 'province', 'Admin2': 'admin2'}, axis=1, inplace=True)
US_data_v2 = US_data.drop(['Lat', 'Long_', 'UID', 'iso2', 'iso3', 'FIPS', 'Country_Region',  'code3'],axis=1)
US_data_row = US_data_v2.sum()

In [3]:
global_data = pd.read_csv('time_series_covid19_confirmed_global.csv')

In [4]:
global_data.rename({'Province/State': 'province', 'Country/Region': 'country'}, axis=1, inplace=True)

In [5]:
global_data = global_data.drop(['Lat','Long'],axis=1)

In [6]:
global_data_v2 = global_data.groupby('country').sum()    #global_data_v2跟global_data的列表已经不一样，global_data用query,而global_data_v2要用loc函数比较好

In [7]:
global_data_v2.loc['America']=US_data_row

In [8]:
global_data_v2

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,55580,55604,55617,55646,55664,55680,55696,55707,55714,55733
Albania,0,0,0,0,0,0,0,0,0,0,...,99062,100246,101285,102306,103327,104313,105229,106215,107167,107931
Algeria,0,0,0,0,0,0,0,0,0,0,...,111764,111917,112094,112279,112461,112622,112805,112960,113092,113255
Andorra,0,0,0,0,0,0,0,0,0,0,...,10672,10699,10712,10739,10775,10799,10822,10849,10866,10889
Angola,0,0,0,0,0,0,0,0,0,0,...,20499,20519,20548,20584,20640,20695,20759,20782,20807,20854
Antigua and Barbuda,0,0,0,0,0,0,0,0,0,0,...,598,598,614,636,646,701,701,726,730,769
Argentina,0,0,0,0,0,0,0,0,0,0,...,2060625,2064334,2069751,2077228,2085411,2093645,2098728,2104197,2107365,2112023
Armenia,0,0,0,0,0,0,0,0,0,0,...,170234,170402,170506,170672,170945,171227,171510,171793,172058,172216
Australia,0,0,0,0,4,5,5,6,9,9,...,28926,28930,28937,28939,28947,28957,28965,28970,28978,28986


In [9]:
#得到对于不同的确诊总数，相应的国家数有多少个
whole_n_countries = []
for i in [1, 10, 100, 1000, 5000, 10000, 20000, 30000, 40000, 50000]:
    n_countries = 0
    for j in range(len(global_data_v2.iloc[:, -1])):
        if global_data_v2.iloc[:, -1][j] > i:
            n_countries +=1
    
    whole_n_countries.append(n_countries)
    
whole_n_countries

[191, 188, 183, 172, 150, 135, 120, 114, 105, 104]

In [10]:
#得到所需确诊病例的国家的index
need_countries=[] 
for k in range(len(global_data_v2.iloc[:, -1])):  
    if global_data_v2.iloc[k, -1] > 20000:
        need_countries.append(global_data_v2.index[k])

In [11]:
len(need_countries)

120

In [12]:
global_data_v3 = global_data_v2.query('country == @need_countries')

In [13]:
global_data_v3

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,55580,55604,55617,55646,55664,55680,55696,55707,55714,55733
Albania,0,0,0,0,0,0,0,0,0,0,...,99062,100246,101285,102306,103327,104313,105229,106215,107167,107931
Algeria,0,0,0,0,0,0,0,0,0,0,...,111764,111917,112094,112279,112461,112622,112805,112960,113092,113255
Angola,0,0,0,0,0,0,0,0,0,0,...,20499,20519,20548,20584,20640,20695,20759,20782,20807,20854
Argentina,0,0,0,0,0,0,0,0,0,0,...,2060625,2064334,2069751,2077228,2085411,2093645,2098728,2104197,2107365,2112023
Armenia,0,0,0,0,0,0,0,0,0,0,...,170234,170402,170506,170672,170945,171227,171510,171793,172058,172216
Australia,0,0,0,0,4,5,5,6,9,9,...,28926,28930,28937,28939,28947,28957,28965,28970,28978,28986
Austria,0,0,0,0,0,0,0,0,0,0,...,443536,445374,446644,448371,450376,452767,454860,457317,459440,460849
Azerbaijan,0,0,0,0,0,0,0,0,0,0,...,232973,233129,233201,233424,233644,233770,233989,234267,234537,234662


In [14]:
global_data_v4 = global_data_v3.loc[:, '5/1/20':'3/1/21']
global_data_v4

,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,...,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,2291,2470,2705,2895,3225,3393,3564,3781,4042,4403,...,55580,55604,55617,55646,55664,55680,55696,55707,55714,55733
Albania,782,789,795,803,820,832,842,850,856,868,...,99062,100246,101285,102306,103327,104313,105229,106215,107167,107931
Algeria,4154,4295,4474,4648,4838,4997,5182,5369,5558,5723,...,111764,111917,112094,112279,112461,112622,112805,112960,113092,113255
Angola,30,35,35,35,36,36,36,43,43,45,...,20499,20519,20548,20584,20640,20695,20759,20782,20807,20854
Argentina,4532,4681,4783,4887,5020,5208,5371,5611,5776,6034,...,2060625,2064334,2069751,2077228,2085411,2093645,2098728,2104197,2107365,2112023
Armenia,2148,2273,2386,2507,2619,2782,2884,3029,3175,3313,...,170234,170402,170506,170672,170945,171227,171510,171793,172058,172216
Australia,6778,6799,6822,6847,6875,6894,6913,6918,6939,6948,...,28926,28930,28937,28939,28947,28957,28965,28970,28978,28986
Austria,15531,15558,15597,15621,15650,15684,15752,15774,15833,15871,...,443536,445374,446644,448371,450376,452767,454860,457317,459440,460849
Azerbaijan,1854,1894,1932,1984,2060,2127,2204,2279,2422,2519,...,232973,233129,233201,233424,233644,233770,233989,234267,234537,234662


In [15]:
#得到每天新增病例的序列
daily_add_all=[]
for k in range(len(global_data_v4)):
    daily_add=[]
    S_da_t = global_data_v4.iloc[k].values
    for i in range(1, len(S_da_t)):
        differ = S_da_t[i] - S_da_t[i-1]
        if differ < 0 or differ == 0:
            differ = 0.00001
            
        daily_add.append(differ)
            
    daily_add_all.append(daily_add)

In [16]:
G_t_all=[]
for k in range(len(daily_add_all)):
    S_t = daily_add_all[k]
    S_t_all = list(map(lambda x: np.log(x), S_t))
    differS_all=[]
    for i in range(1, len(S_t_all)):
        differS = S_t_all[i] - S_t_all[i-1]
        if differS == 0 or differS < 0:
            differS = 0.00001
            
        differS_all.append(differS)
            
    G_t_all.append(differS_all)

In [17]:
Cij_all=[]
for i in range(len(G_t_all)):
    Cij_row=[]
    for j in range(len(G_t_all)):
        sdi = np.sqrt(np.mean(list(map(lambda x: pow(x, 2), G_t_all[i]))) - pow(np.mean(G_t_all[i]), 2))
        sdj = np.sqrt(np.mean(list(map(lambda x: pow(x, 2), G_t_all[j]))) - pow(np.mean(G_t_all[j]), 2))
        Cij = (np.mean([a*b for a,b in zip(G_t_all[i], G_t_all[j])]) - np.mean(G_t_all[i]) * np.mean(G_t_all[j])) / (sdi * sdj)
        Cij_row.append(Cij)
        
    Cij_all.append(Cij_row)
    
Cij_all = np.array(Cij_all)

In [18]:
mat = np.matrix(Cij_all)
with open('outfile_dailyAdd_L20000.txt','wb') as f:
    for line in mat:
        np.savetxt(f, line, fmt='%.2f')